In [ ]:
#Please run the first 3 cells to have all things to work
#The fourth is as substitution you won't need it for now, but you may 
#need if you want to delete log of tensorboard

#Then the rest is the same just changing the name of environment


!pip install stable-baselines
#!pip install --upgrade pip
!pip install gym[box2d]
!pip install -c conda-forge pyglet
#remove " > /dev/null 2>&1" to see what is going on under the hood
!apt-get install swig cmake libopenmpi-dev zlib1g-dev xvfb x11-utils ffmpeg -qq #remove -qq for full output
!pip install stable-baselines[mpi] box2d box2d-kengz pyvirtualdisplay pyglet==1.3.1 --quiet #remove --quiet for full output 
# Stable Baselines only supports tensorflow 1.x for now
#%tensorflow_version 1.x

import imageio
import base64
import IPython
import PIL.Image
import pyvirtualdisplay
#Video stuff 
from pathlib import Path
from IPython import display as ipythondisplay

In [ ]:
#!sudo apt-get update && sudo apt-get install cmake libopenmpi-dev zlib1g-dev
!pip install stable-baselines[mpi]

%tensorflow_version 1.x
#import tensorflow as tf

#!apt-get install ffmpeg freeglut3-dev xvfb  # For visualization
!pip install stable-baselines[mpi]==2.10.0

#!pip install gym
#!pip install gym[atari]

#import os
#os.kill(os.getpid(), 9)

import stable_baselines
stable_baselines.__version__

import gym
import numpy as np

In [3]:
#Load the TensorBoard notebook extension
%load_ext tensorboard
import tensorflow as tf
import datetime, os

# Rendering

In [4]:
# Stable Baselines only supports tensorflow 1.x for now
%tensorflow_version 1.x
!apt-get update
!apt-get install swig cmake libopenmpi-dev zlib1g-dev ffmpeg freeglut3-dev xvfb
##!pip install stable-baselines[mpi] --upgrade
!pip install stable-baselines
!pip install pybullet
!pip install box2d box2d-kengz pyyaml pytablewriter optuna scikit-optimize

Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:6 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Ign:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Fetched 252 kB in 2s (156 kB/s)
Reading package lists... Done
Reading package lists... Done
Building

In [5]:
import imageio
import base64
import IPython
import PIL.Image
import pyvirtualdisplay
#Video stuff 
from pathlib import Path
from IPython import display as ipythondisplay

def show_videos(video_path='', prefix=''):
  html = []
  for mp4 in Path(video_path).glob("{}*.mp4".format(prefix)):
      video_b64 = base64.b64encode(mp4.read_bytes())
      html.append('''<video alt="{}" autoplay 
                    loop controls style="height: 400px;">
                    <source src="data:video/mp4;base64,{}" type="video/mp4" />
                </video>'''.format(mp4, video_b64.decode('ascii')))
  ipythondisplay.display(ipythondisplay.HTML(data="<br>".join(html)))
  
import os
os.system("Xvfb :1 -screen 0 1024x768x24 &")
os.environ['DISPLAY'] = ':1'

In [6]:
#!sudo apt-get update && sudo apt-get install cmake libopenmpi-dev zlib1g-dev
#!pip install stable-baselines[mpi]

%tensorflow_version 1.x
#import tensorflow as tf

#!apt-get install ffmpeg freeglut3-dev xvfb  # For visualization
#!pip install stable-baselines[mpi]==2.10.0

#!pip install gym
#!pip install gym[atari]

#import os
#os.kill(os.getpid(), 9)

import stable_baselines
stable_baselines.__version__

import gym
import numpy as np

In [ ]:
%tensorflow_version 1.x
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)


try:
  import optuna
except ImportError:
  !pip install optuna
  import optuna
import gym
import os
import numpy as np
import matplotlib.pyplot as plt

from stable_baselines import PPO2, DQN
from stable_baselines.common.vec_env import VecVideoRecorder, DummyVecEnv
from stable_baselines.common.evaluation import evaluate_policy
from stable_baselines.common.cmd_util import make_vec_env

def optimize_ppo2(trial):
    """ Learning hyperparamters we want to optimise"""
    return {
        'n_steps': int(trial.suggest_loguniform('n_steps', 16, 2048)),
        'gamma': trial.suggest_loguniform('gamma', 0.9, 0.9999),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1.),
        'ent_coef': trial.suggest_loguniform('ent_coef', 1e-8, 1e-1),
        'cliprange': trial.suggest_uniform('cliprange', 0.1, 0.4),
        'noptepochs': int(trial.suggest_loguniform('noptepochs', 1, 48)),
        'lam': trial.suggest_uniform('lam', 0.8, 1.)
    }


def optimize_agent(trial):
    """ Train the model and optimize
        Optuna maximises the negative log likelihood, so we
        need to negate the reward here
    """
    # Create log dir
    log_dir = "tmp/"
    os.makedirs(log_dir, exist_ok=True)

    #env_name='CartPole-v1'
    env_name='LunarLander-v2'
    #env_name='MountainCar-v0'
    #env_name='Acrobot-v1'
    model_params = optimize_ppo2(trial)

    #env = make_vec_env(lambda: gym.make(env_name), n_envs=16, seed=0)
    env = gym.make(env_name)
    env = DummyVecEnv([lambda: env])
    
    model = PPO2('MlpPolicy', env, verbose=0, nminibatches=1, **model_params,
                 tensorboard_log="./ppo/")
    #model = PPO('MlpPolicy', env, verbose=0, **model_params)
    
    time_steps=600000
    model.learn(total_timesteps=int(time_steps))


    mean_reward, _ = evaluate_policy(model, gym.make(env_name), n_eval_episodes=10)
    #print(model.)

    !rm -rf video
    #obs = env.reset()
    length = 100000
    env=VecVideoRecorder(env, video_folder='./video',
                                record_video_trigger=lambda step: step == 0, video_length=length,
                                name_prefix='abc')
    obs = env.reset()
    for i in range(length):
      action, _states = model.predict(obs)
      obs, rewards, done, info = env.step(action)
      if done:
        break
      #print("here2")
      #env.render()

    #print("here3")
    env.close()
    

    return mean_reward


if __name__ == '__main__':
    study = optuna.create_study(direction="maximize",pruner=optuna.pruners.SuccessiveHalvingPruner())
    try:
        study.optimize(optimize_agent, n_trials=2, n_jobs=2,)
        
        #%tensorboard --logdir ./ppo/
    except KeyboardInterrupt:
        print('Interrupted by keyboard.')

[I 2020-12-14 05:05:07,936] A new study created in memory with name: no-name-d773db44-47ce-484f-af4a-33b46d3e8e17
[I 2020-12-14 05:36:56,171] Trial 0 finished with value: -133.90722672467132 and parameters: {'n_steps': 870.2832258576916, 'gamma': 0.906278247308756, 'learning_rate': 0.5018536936835667, 'ent_coef': 9.138625077453658e-05, 'cliprange': 0.32601010006330855, 'noptepochs': 2.681673432317369, 'lam': 0.8261842718933067}. Best is trial 0 with value: -133.90722672467132.


In [ ]:
show_videos('video','abc0-step-0-to-step-80000')